In [11]:
from ortools.linear_solver import pywraplp
import pandas as pd

solver = pywraplp.Solver.CreateSolver('CP_SAT')

In [12]:
dfg = pd.read_excel("inputs.xlsx", sheet_name = "gen")
dfd = pd.read_excel("inputs.xlsx", sheet_name = "load")

In [13]:
dfg

,id,limit,cost
0,0,20,0.10
1,1,10,0.05
2,2,40,0.30
3,3,50,0.40
4,4,5,0.01


In [14]:
dfd

,id,value
0,0,50
1,1,20
2,2,30


In [22]:
pg = {}
cg = {}
ig= {}

M = 1000

for i in dfg['id']:
    pg[i] = solver.NumVar(0, 100, 'pg_%i' %(i))
    ig[i] = solver.IntVar(0, 1, 'ig_%i' %(i))

for i in dfg['id']:
    solver.Add(pg[i] <= dfg['limit'][i])
    solver.Add(M*ig[i] >= pg[i])

solver.Add(pg[0] + pg[3] >= dfd['value'][0])

solver.Add(sum(pg[i] for i in dfg['id']) == sum(dfd['value']))

solver.Minimize(sum(pg[i]*dfg['cost'][i] for i in dfg['id']))

In [19]:
results = solver.Solve()

if results==pywraplp.Solver.OPTIMAL: print('Optimal Found')

Optimal Found


In [20]:
for i in dfg['id']:
    print(i, pg[i].solution_value())

0 20.0
1 10.0
2 35.0
3 30.0
4 5.0
